## Google Drive setup

In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [2]:
# %cd "/content/drive/MyDrive/colab_not"  

## Local setup

In [8]:
import os
import sys
from os import path, pardir
import pathlib

# Add the "scripts" folder to the PATH
scripts = path.join(pathlib.Path(os.path.abspath('')), 'scripts')
if scripts not in sys.path:
    sys.path.append(scripts)

# Import our libraries
from helpers import data_path
import data_cleaning as dc

## Project II

In [34]:
import tensorflow as tf
from keras.layers.experimental.preprocessing import TextVectorization
from keras.regularizers import l2
import numpy as np
import string
import pandas as pd

ModuleNotFoundError: No module named 'keras'

In [10]:
def split_set(data_to_split, ratio=0.8):
    mask = np.random.rand(len(data_to_split)) < ratio
    return [data_to_split[mask].reset_index(drop=True), data_to_split[~mask].reset_index(drop=True)]

In [11]:
batch_size = 64

raw_train_pos = pd.read_table("data/train/train_pos_clean.txt", sep = "\n", header=None,quoting= 3)
raw_train_pos['pred'] = 1

raw_train_neg = pd.read_table("data/train/train_neg_clean.txt", sep = "\n", header=None, quoting = 3)
raw_train_neg['pred'] = 0


#########################################################################
########### TESTING ONLY - Take only 25% of the whole dataset ###########
#########################################################################
# pos_total = len(raw_train_pos)
# neg_total = len(raw_train_neg)
# raw_train_pos = raw_train_pos[:int(pos_total * .25)]
# raw_train_neg = raw_train_neg[:int(neg_total * .25)]
#########################################################################
#########################################################################
#########################################################################

# Form training data and remove duplicates
raw_train = pd.concat((raw_train_neg,raw_train_pos))
raw_train.drop_duplicates(inplace=True)

# Separating training and validation data
raw_train_tr,raw_train_val  = split_set(raw_train.sample(frac=1,random_state=0))
target_train_tr = raw_train_tr.pop('pred')
target_train_val = raw_train_val.pop('pred')

# Turning Pandas dataframes into Tensorflow datasets
raw_train_ds = tf.data.Dataset.from_tensor_slices((np.squeeze(raw_train_tr.values), target_train_tr.values))
raw_val_ds = tf.data.Dataset.from_tensor_slices((np.squeeze(raw_train_val.values),target_train_val.values))


# Batchify data
raw_train_ds = raw_train_ds.batch(batch_size=batch_size)
raw_val_ds = raw_val_ds.batch(batch_size=batch_size)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)

Number of batches in raw_train_ds: 2257
Number of batches in raw_val_ds: 565


In [12]:
def remove_skips(input_data):
    return tf.strings.regex_replace(input_data, "\n", " ")

In [13]:
# Keep only text
text_ds = raw_train_ds.map(lambda x, y: x) 

In [14]:
type(text_ds)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [15]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [17]:
max_features = len(raw_train)
embedding_dim = 32
sequence_length = 280

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

vectorize_layer.adapt(text_ds)

# We want to know our vacobulary size, so that we can pass it to the embedding layer
vocab_size = len(vectorize_layer.get_vocabulary())

In [18]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
# test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
# test_ds = test_ds.cache().prefetch(buffer_size=10)

In [35]:
model = keras.Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=sequence_length))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 280, 32)           2846208   
_________________________________________________________________
flatten_1 (Flatten)          (None, 8960)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8961      
Total params: 2,855,169
Trainable params: 2,855,169
Non-trainable params: 0
_________________________________________________________________
None


Importing the test data.

In [36]:
epochs = 4
# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/4
2257/2257 [==============================] - 46s 20ms/step - loss: 0.5323 - accuracy: 0.7566 - val_loss: 0.4960 - val_accuracy: 0.7725
Epoch 2/4
2257/2257 [==============================] - 45s 20ms/step - loss: 0.4556 - accuracy: 0.8081 - val_loss: 0.4737 - val_accuracy: 0.7873
Epoch 3/4
2257/2257 [==============================] - 43s 19ms/step - loss: 0.4262 - accuracy: 0.8258 - val_loss: 0.4677 - val_accuracy: 0.7905
Epoch 4/4
2257/2257 [==============================] - 45s 20ms/step - loss: 0.4061 - accuracy: 0.8372 - val_loss: 0.4660 - val_accuracy: 0.7912


In [292]:
# test_ds = tf.data.TextLineDataset("data/test/test_data_cl.txt")

In [41]:
def vectorize_text_test(text):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)

test_ds = test_ds.map(vectorize_text_test)

test_ds = test_ds.cache().prefetch(buffer_size=10)

In [42]:
preds = model.predict(test_ds)

In [43]:
preds = np.where(preds > 0.5, 1, -1)

In [46]:
preds_path = "predictions/preds.csv"
with open(preds_path, "w") as f:
    f.write("Id,Prediction\n")
    for i, y in enumerate(preds):
        f.write(str(i + 1) + "," + str(int(y)) + "\n")
print(f"Done. Predictions can be found in {preds_path}.")

Done. Predictions can be found in predictions/preds.csv.
